In [1]:
import requests
from bs4 import BeautifulSoup
import json
import datetime
import os
import sys
import time
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import pickle
import nltk
import re
import jieba
import jieba.analyse
from optparse import OptionParser
import matplotlib.pyplot as plt 
import random
import matplotlib
from matplotlib import rcParams # 中文設定1: 楷體字
rcParams["font.sans-serif"] = ["kaiu"] 
rcParams["font.family"] ="DFKai-sb"
matplotlib.use('TkAgg')
matplotlib.rcParams['font.sans-serif'] = ['SimHei'] #组件字体
import tkinter as tk
window = tk.Tk()

path='C:/My python/'
driver_address = 'C:/My python/chromedriver.exe'

#建立時間的條件，用來篩選出今天盤後到明天盤前的資料
nowtime = datetime.datetime.now() #現在時間
yesterday2pm = (datetime.datetime.now() + datetime.timedelta(-1)).replace(hour=14, minute=0, second=0, microsecond=0)
today9am = datetime.datetime.now().replace(hour=9, minute=0, second=0, microsecond=0)
tomorrow12am = (datetime.datetime.now() + datetime.timedelta(+1)).replace(hour=0, minute=0, second=0, microsecond=0)
today2pm = datetime.datetime.now().replace(hour=14, minute=0, second=0, microsecond=0)
today12am = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
today12pm = datetime.datetime.now().replace(hour=12, minute=0, second=0, microsecond=0)
yesterday12pm = (datetime.datetime.now() + datetime.timedelta(-1)).replace(hour=12, minute=0, second=0, microsecond=0)

stopwords= ['台股','台泥','台股','亞泥','嘉泥','環泥','幸福','信大','東泥','味全','味王','大成','大飲','卜蜂','統一','愛之味','泰山','福壽','台榮','福懋油','佳格','聯華','聯華食','大統益','天仁','黑松','興泰','宏亞','鮮活果汁-KY','台塑','南亞','台聚','華夏','三芳','亞聚','台達化','台苯','國喬','聯成','中石化','達新','上曜','東陽','大洋','永裕','地球','恆大','台化','再生-KY','廣華-KY','昭輝','勝悅-KY','富林-KY','八貫','遠東新','新纖','南染','宏洲','東和','廣豐','嘉裕','東華','新紡','利華','大魯閣','福懋','中福','華友聯','勤益控','三地開發','中和','南紡','大東','名軒','立益','力麗','大宇','宏和','力鵬','佳和','年興','宏益','大將','台富','集盛','怡華','宜進','聯發','宏遠','強盛','得力','偉全','聚隆','南緯','昶和','大統新創','首利','三洋紡','台南','弘裕','業旺','儒鴻','聚陽','士電','東元','正道','永大','瑞利','中興電','亞力','力山','川飛','利奇','華城','大億','堤維西','耿鼎','江申','日馳','鑽全','恩德','樂士','亞崴','高林股','勤美','車王電','中宇','和大','廣隆','正峰','巨庭','喬福','錩泰','伸興','中砂','倉佑','信錦','程泰','吉茂','永冠-KY','亞德客-KY','英瑞-KY','岱宇','華電','聲寶','華新','華榮','大亞','中電','宏泰','三洋電','大山','億泰','榮星','合機','艾美特-KY','中化','南僑','葡萄王','東鹼','和益','東聯','永光','興農','國化','和桐','長興','中纖','生達','三晃','台肥','中碳','台硝','元禎','永記','中華化','花仙子','美吾華','毛寶','五鼎','杏輝','日勝化','喬山','臺鹽','寶齡富錦','中化生','勝一','展宇','和康生','科妍','神隆','美時','台玻','寶徠','冠軍','潤隆','中釉','和成','凱撒衛','士紙','正隆','華紙','寶隆','永豐餘','榮成','中鋼','東和鋼鐵','燁興','高興昌','第一銅','春源','春雨','中鋼構','中鴻','豐興','官田鋼','美亞','聚亨','燁輝','志聯','千興','大成鋼','威致','盛餘','彰源','新光鋼','新鋼','佳大','允強','海光','上銀','川湖','橋椿','運錩','南港','泰豐','台橡','國際中橡','正新','建大','厚生','南帝','華豐','鑫永銓','六暉-KY','裕隆','中華','三陽工業','和泰車','台船','裕日車','劍麟','為升','宇隆','百達-KY','英利-KY','艾姆勒','宏旭-KY','汎德永業','光寶科','麗正','聯電','全友','台達電','金寶','華通','台揚','楠梓電','鴻海','東訊','中環','仁寶','國巨','廣宇','華泰','台積電','精英','友訊','旺宏','光罩','光磊','茂矽','華邦電','智邦','聯強','海悅','錸德','順德','佳世達','宏碁','鴻準','敬鵬','英業達','華碩','廷鑫','所羅門','致茂','藍天','矽統','倫飛','昆盈','燿華','金像電','菱生','大同','震旦行','佳能','凱美','技嘉','微星','瑞昱','虹光','廣達','台光電','群光','精元','威盛','云辰','正崴','億光','研華','友通','映泰','凌陽','毅嘉','漢唐','浩鑫','國碩','南亞科','友達','中華電','環科','精技','錩新','圓剛','仲琦','新巨','建準','固緯','隴華','承啟','鼎元','三商電','興勤','銘旺科','燦坤','聯昌','互盛電','統懋','偉詮電','翔耀','美律','太空梭','超豐','新美齊','億麗','兆勁','晶電','京元電子','神腦','創見','凌群','聯發科','全新','奇力新','飛宏','義隆','敦吉','建通','光群雷','良得電','盟立','麗臺','冠西電','志聖','華經','資通','立隆電','可成','鉅祥','美隆電','大毅','敦陽科','強茂','連宇','百容','希華','兆赫','一詮','漢平','瑞軒','吉祥全','華新科','揚博','普安','卓越','怡利電','宏達電','國建','國產','國揚','太設','全坤建','太子','龍邦','中工','新建','冠德','京城','宏璟','皇普','華建','宏盛','達欣工','宏普','聯上發','基泰','櫻花建','愛山林','興富發','皇昌','皇翔','根基','日勝生','華固','潤弘','益航','長榮','新興','裕民','榮運','嘉里大榮','陽明','華航','志信','中航','中櫃','東森','萬海','山隆','台航','長榮航','亞航','台灣高鐵','漢翔','台驊投控','慧洋-KY','宅配通','萬企','華園','國賓','六福','第一店','晶華','遠雄來','夏都','美食-KY','王品','雄獅','寒舍','雲品','彰銀','京城銀','台中銀','旺旺保','華票','中壽','台產','臺企銀','高雄銀','聯邦銀','台開','遠東銀','安泰銀','新產','中再保','第一保','統一證','三商壽','華南金','富邦金','國泰金','開發金','玉山金','元大金','兆豐金','台新金','新光金','國票金','永豐金','中信金','第一金','王道銀行','欣欣','遠百','匯僑','三商','高林','特力','統領','麗嬰房','統一超','農林','潤泰全','鼎固-KY','淘帝-KY','客思達-KY','凱羿-KY','歐格','健和興','豐達科','神基','晶豪科','大立光','華立','今皓','晟銘電','聯陽','全漢','嘉晶','奇鋐','同開','亞光','鴻名','威強電','信邦','憶聲','星通','禾伸堂','盛達','增你強','零壹','德律','佰鴻','偉訓','威健','聯詠','智原','文曄','欣興','全台','遠見','揚智','晶技','科風','健鼎','台灣大','建碁','訊舟','益登','和鑫','鈺德','力特','夆典','立萬利','蔚華科','總太','喬鼎','立德','華晶科','銘異','建漢','日電貿','聯傑','一零四','正達','景岳','大量','景碩','全科','晟鈦','緯創','虹冠電','昇陽','勝德','昇貿','聯德','閎暉','弘憶股','同泰','泰碩','麗清','奇偶','新日興','明泰','新世紀','玉晶光','京鼎','融程電','譁裕','台端','榮創','創意','聯鈞','晶睿','群創','誠研','維熹','揚明光','華擎','柏騰','安馳','晶相光','台勝科','嘉澤','晶彩科','誠創','州巧','敦泰','聯穎','嘉威','牧德','聯合再生','辛耘','通嘉','艾笛森','力銘','智易','宏致','谷崧','碩天','洋華','達邁','健策','世芯-KY','貿聯-KY','圓展','TPK-KY','新至陞','亞太電','達能','海華','隆達','大眾控','大聯大','欣陸','合勤控','永信','神達','上緯投控','日月光投控','永崴投控','佳醫','雃博','懷特','旭富','亞諾法','麗豐-KY','龍燈-KY','國光生','全宇生技-KY','訊映','承業醫','佐登-KY','炎洲','如興','利勤','廣越','冠星-KY','東台','瑞智','拓凱','全球傳動','銘鈺','智伸科','力達-KY','氣立','永新-KY','強信-KY','穎漢','元翎','時碩工業','鈞興-KY','駐龍','大銀微系統','光隆精密-KY','德淵','國精化','信昌化','華廣','康普','台耀','三福化','材料-KY','雙鍵','南寶','日成-KY','遠傳','正文','聯德控股-KY','致伸','事欣科','新唐','泰鼎-KY','燦星網','太極','茂林-KY','和碩','嘉彰','康控-KY','凌通','光鋐','臻鼎-KY','誠美材','天鈺','十銓','立積','佳凌','眾達-KY','榮科','傳奇','鑫禾','三星','訊連','科嘉-KY','東科-KY','達興材料','乙盛-KY','虹堡','鎧勝-KY','祥碩','禾聯碩','jpp-KY','界霖','豐祥-KY','中磊','崇越','瀚宇博','松翰','慧友','建國','隆大','工信','遠雄','順天','鄉林','皇鼎','長虹','東明-KY','永固-KY','遠雄港','四維航','鳳凰','中租-KY','上海商銀','合庫金','台南-KY','大洋-KY','群益證','群益期','競國','聚碩','鎰勝','彩晶','迎廣','達運','上福','鈞泰','金橋','富爾特','亞翔','柏承','友勁','百一','嘉聯益','鈞寶','華興','捷泰','凌華','宏齊','互億','瑞儀','達麗','關貿','大豐電','豐藝','精成科','巨路','帆宣','佳必琪','亞弘電','盛群','詮欣','飛捷','今國光','聯茂','精誠','和椿','居易','聚鼎','天瀚','光鼎','超眾','華孚','力成','迅杰','定穎','矽格','台郡','同欣電','宏正','台表科','全國電','康舒','淳安','啟碁','聯嘉','華上','悅城','旭隼','群電','樺漢','矽力-KY','瑞祺電通','光麗-KY','光聖','元晶','鈺邦','訊芯-KY','康友-KY','GIS-KY','台數科','安集','晶碩','南六','台塑化','捷敏-KY','愛普','晶心科','泰福-KY','易華電','興能高','虹揚-KY','研揚','鋼聯','申豐','動力-KY','和潤企業','ABC-KY','帝寶','必應','基士德-KY','科定','羅麗芬-KY','中揚光','緯穎','復盛應用','三能-KY','騰輝電子-KY','鋐寶科技','旭暉應材','惠特','嘉基','匯僑設計','台通','矽創','尖點','昇陽半導體','雷虎','台虹','南電','長華*','陞泰','致新','華冠','瀚荃','錸寶','凌巨','華東','至上','振樺電','福懋科','南茂','達方','無敵','勤誠','志超','明基材','寶一','菱光','富鼎','宇瞻','日友','建新國際','羅昇','百和興業-KY','福貞-KY','可寧衛','基勝-KY','金麗-KY','威宏-KY','阿瘦','富邦媒','柏文','潤泰材','億豐','美吉吉-KY','波力-KY','山林水','東哥遊艇','泰昇-KY','政伸','商億-KY','吉源-KY','鼎炫-KY','台汽電','新天地','高力','鈺齊-KY','台火','寶成','大華','欣巴巴','統一實','大台北','豐泰','櫻花','偉聯','美利達','中保科','欣天然','康那香','巨大','福興','新保','新海','泰銘','中視','秋雨','中聯資源','欣高','中鼎','成霖','慶豐富','全國','百和','宏全','信義','裕融','茂順','好樂迪','新麗','潤泰新','三發地產','佳龍','世紀鋼']








def run_TAIEX():   #x代表要輸入的股票
    # from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
    # from matplotlib.figure import Figure    
    def cn(): #中時爬蟲===>中時新聞只能應用在加權指數預測、因為他沒有個股新聞只有大盤
        cn_title_list = []
        cn_time_list = []
        cn_address_list = []
        final_list = []
        for w in range(1,5):
            page_address = "https://www.chinatimes.com/money/securities?page="+str(w)
            cnews_url=page_address                       #根據for迴圈建構要爬的網址
            re = requests.post(url=cnews_url) #呼叫 煮湯流程
            soup = BeautifulSoup(re.text,"html.parser") #煮湯流程
            cnews_adress = soup.find_all('h3',{'class':'title'})   #煮湯流程       #建構list方便儲存每個新聞的網址
            for i in cnews_adress:
                i1 = "https://www.chinatimes.com"+i.a["href"] #i1代表每一個新聞的網址 
                cn_address_list.append(i1) #放入列表
            for i in cnews_adress:
                text = i.text
                if text[0] =="《":
                    cn_title_list.append(text[text.index("》")+1:])
                else:
                    cn_title_list.append(i.text)

            for x in cn_address_list: #針對每頁中的個別新聞爬取內文
                cnews_url=x #x=個別新聞的網址
                re = requests.post(url=cnews_url)#煮湯流程
                soup = BeautifulSoup(re.text,"html.parser") #煮湯流程
                cnews_content_time = soup.find('time')#煮湯流程
                time_content = cnews_content_time['datetime']
                time_content_final = datetime.datetime.strptime(time_content,'%Y-%m-%d %H:%M')
                cn_time_list.append(time_content_final)

            for n,t in zip(cn_title_list,cn_time_list):
                if 0 < nowtime.weekday() < 4 :
                    if today12pm < nowtime < tomorrow12am and t > today12pm :
                        final_list.append(n)
                    elif today12am < nowtime < today12pm and t > yesterday12pm :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 0 and nowtime > today12pm :
                    if today12pm < nowtime < tomorrow12am and t > today12pm :
                        final_list.append(n)
                    elif today12am < nowtime < today12pm and t > yesterday12pm :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 4 and nowtime < today12pm :
                    if today12pm < nowtime < tomorrow12am and t > today12pm :
                        final_list.append(n)
                    elif today12am < nowtime < today12pm and t > yesterday12pm :
                        final_list.append(n)
                    else :
                        continue
                else :
                    if nowtime.weekday() == 4 and nowtime > today12pm :
                        if t > today12pm :
                            final_list.append(n)
                        else :
                            continue
                    elif nowtime.weekday() == 0 and nowtime < today12pm :
                        if t > today12pm-datetime.timedelta(days = 3) :
                            final_list.append(n)
                        else :
                            continue
                    elif nowtime.weekday() == 5 :
                        if t > yesterday12pm :
                            final_list.append(n)
                        else :
                            continue
                    elif nowtime.weekday() == 6 :
                        if t > yesterday12pm-datetime.timedelta(days = 1) :
                            final_list.append(n)
                        else :
                            continue
                    else :
                            continue
            if final_list == [] :
                final_list.append("上漲")
                final_list.append("下跌")
            return final_list
        
    def yahoo(x): #Yahoo爬蟲===>yahoo沒有大盤新聞專頁、因此只能用權值股預測大盤，x輸入權值股即可，另外yahoo可以應用於個股查詢功能。
        yahoo_title_list= []
        yahoo_time_list = []
        yahoo_address_list = []
        final_list = []

        for w in range(1,3): 
            url_yahoo = "https://tw.stock.yahoo.com/q/h?s="+x+"&pg="+ str(w) #str(s)代表搜尋欄位 str(w)代表頁數
            re = requests.get(url = url_yahoo)
            soup_address = BeautifulSoup(re.text,"html.parser") 
            yahoo_address = soup_address.find_all("td",{"height":"37"})

            for i in yahoo_address:
                text = i.text
                if text[0] =="《":
                    yahoo_title_list.append(text[text.index("》")+1:])
                else:            
                    yahoo_title_list.append(i.text)
                


            for i in yahoo_address:
                k = "https://tw.stock.yahoo.com/"+i.a["href"]
                yahoo_address_list.append(k) #第w頁的所有標題網址

        for x in yahoo_address_list:  #第w頁第i個標題網址
            re = requests.get(url =x)
            soup_content = BeautifulSoup(re.text,"html.parser")
            yahoo_time = soup_content.find("time",{"class":"caas-attr-meta-time"})  #上傳時間
            yahoo_time = str(yahoo_time["datetime"]).replace("T", " ").replace(".000Z", "")
            # debug
            if yahoo_time == None:
                continue
            # 
            yahoo_time = datetime.datetime.strptime(yahoo_time, "%Y-%m-%d %H:%M:%S")+datetime.timedelta(hours = 8)
            yahoo_time_list.append(yahoo_time)

        for n,t in zip(yahoo_title_list,yahoo_time_list):
            if 0 < nowtime.weekday() < 4 :
                if today12pm < nowtime < tomorrow12am and t > today12pm :
                    final_list.append(n)
                elif today12am < nowtime < today12pm and t > yesterday12pm :
                    final_list.append(n)
                else :
                    continue
            elif nowtime.weekday() == 0 and nowtime > today12pm :
                if today12pm < nowtime < tomorrow12am and t > today12pm :
                    final_list.append(n)
                elif today12am < nowtime < today12pm and t > yesterday12pm :
                    final_list.append(n)
                else :
                    continue
            elif nowtime.weekday() == 4 and nowtime < today12pm :
                if today12pm < nowtime < tomorrow12am and t > today12pm :
                    final_list.append(n)
                elif today12am < nowtime < today12pm and t > yesterday12pm :
                    final_list.append(n)
                else :
                    continue
            else :
                if nowtime.weekday() == 4 and nowtime > today12pm :
                    if t > today12pm :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 0 and nowtime < today12pm :
                    if t > today12pm-datetime.timedelta(days = 3) :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 5 :
                    if t > yesterday12pm :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 6 :
                    if t > yesterday12pm-datetime.timedelta(days = 1) :
                        final_list.append(n)
                    else :
                        continue
                else :
                        continue
        if final_list == [] :
            final_list.append("上漲")
            final_list.append("下跌")
        return final_list 


    def eco(x): #經濟日報不論要應用在大盤預測或是個股搜索都可以，若要預測大盤，則x輸入"大盤" 和其他權值股。
        eco_address_list = []
        eco_time_list = []
        eco_title_list = []
        for w in range(1,3): #先爬出搜尋2330的所有新聞網址 (先假設爬五頁)
            url_eco = "https://money.udn.com/search/result/1001/"+x+"/"+str(w)

            headers = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
                    "cookie":'_ga=GA1.2.927708406.1601618986; _fbp=fb.1.1601618985824.706528271; _ss_pp_id=b3c8335fbeb19f9adaf1601590188761; dable_uid=53822174.1601061552644; __dtsu=4C30160291899678DF25A79BB792ADBD; truvid_protected={"val":"c","level":2,"geo":"TW","timestamp":1605071289}; __auc=04d5a0b6175f83e7d1a75f55ca6; __erEvntUid=noset; __eruid=d8400cdf-4c38-c641-a4d9-87f2d35c2f8a; _fbc=fb.1.1606187751623.IwAR32rqYXY1HwRAL_qIat0HFknrbrsfzraHTaRIhOYM6tqU0r5y3hnMNNWi4; __BWfp=c1606187759520x1a75e6b90; _gid=GA1.2.630643660.1607095760; __gads=ID=56b9d1fd37dbdf4b:T=1607095770:S=ALNI_MZoFn8UvghJvOmXcQR4TUlpiIJPOg; _gliaplayer_user_info={%22city%22:%22shibuya%20city%22%2C%22uid%22:%22a96b4d50-0182-11eb-9ff0-0fb12ecbca9d%22%2C%22country%22:%22TW%22%2C%22region%22:%2213%22%2C%22source%22:%22CF%22%2C%22latlong%22:%2235.661971%2C139.703795%22%2C%22ip%22:%2261.58.108.239%22}; meter_expires=Fri, 11 Dec 2020 17:55:35 GMT; ucfunnel_uid=ad86a035-a317-3d62-8366-fb6443c6f307; __retuid=1a903bd2-775a-d99b-c37a-65052f36a210; csrf_cookie_name=afd2db682c0216bd0166ddd5f541e09b; __asc=2d7132f217631ad0c388694c5f4; gliaplayer_ssid=8c31cab0-36c6-11eb-8837-29e472ee0491; meter_articleIds={"5066965":1,"5068548":1}; meter_day_read_pages={"20201205":2}; meter_limit_day7=2; meter_limit_day1=2; meter_1=2; meter_7=2; _td=f7597bf8-e560-44e6-b888-c97d831c391a; __retfs=fSes-866ff7c9-b611-95f-11d2; __lastv=0'}

            re = requests.get(url = url_eco, headers = headers )

            soup_address = BeautifulSoup(re.text,"html.parser") 
        #     eco_address = soup_address.find_all(href=r.compile("https://money.udn.com/money/story"))
            eco_title = soup_address.find("div",id="search_content")
            eco_title = str(eco_title)
            soup_title = BeautifulSoup(eco_title,"html.parser")
            eco_address = soup_title.find_all("a")
            eco_title1 = soup_title.find_all("h3")

            for n in eco_title1:
                eco_title_list.append(n.text)
            for i in eco_address:
                eco_address_list.append(i["href"])

        for x in eco_address_list:  #根據列表中的網址爬取標題
            re = requests.get(url = x, headers = headers )
            soup_time = BeautifulSoup(re.text,"html.parser")
            soup_time1 = soup_time.find("div",{"class":"shareBar__info"})
            soup_time2 = soup_time1.text[:17].replace("\n","")

            soup_time3 = datetime.datetime.strptime(soup_time2,'%Y-%m-%d %H:%M')
            eco_time_list.append(soup_time3)


        final_list = []

        for n,t in zip(eco_title_list,eco_time_list):
                if 0 < nowtime.weekday() < 4 :
                    if today12pm < nowtime < tomorrow12am and t > today12pm :
                        final_list.append(n)
                    elif today12am < nowtime < today12pm and t > yesterday12pm :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 0 and nowtime > today12pm :
                    if today12pm < nowtime < tomorrow12am and t > today12pm :
                        final_list.append(n)
                    elif today12am < nowtime < today12pm and t > yesterday12pm :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 4 and nowtime < today12pm :
                    if today12pm < nowtime < tomorrow12am and t > today12pm :
                        final_list.append(n)
                    elif today12am < nowtime < today12pm and t > yesterday12pm :
                        final_list.append(n)
                    else :
                        continue
                else :
                    if nowtime.weekday() == 4 and nowtime > today12pm :
                        if t > today12pm :
                            final_list.append(n)
                        else :
                            continue
                    elif nowtime.weekday() == 0 and nowtime < today12pm :
                        if t > today12pm-datetime.timedelta(days = 3) :
                            final_list.append(n)
                        else :
                            continue
                    elif nowtime.weekday() == 5 :
                        if t > yesterday12pm :
                            final_list.append(n)
                        else :
                            continue
                    elif nowtime.weekday() == 6 :
                        if t > yesterday12pm-datetime.timedelta(days = 1) :
                            final_list.append(n)
                        else :
                            continue                    
                    else :
                            continue
        if final_list == [] :
            final_list.append("上漲")
            final_list.append("下跌")
        return final_list

    def cm_TWA00_u():
        url="https://www.cmoney.tw/follow/channel/getdata/articlelistofstockv2?articleCategory=Personal&channelId=103681&size=21&sTime=&articleSortType=latest&articleSortCount=0&isIncludeLimitedAskArticle=false&_=1608325778808"
        return url
    def cm_TWA00_d():
        url_d = "https://www.cmoney.tw/follow/channel/stock-TWA00?chart=d&type=Personal"
        return url_d
    def cm_2454_u():
        url="https://www.cmoney.tw/follow/channel/getdata/articlelistofstockv2?articleCategory=Personal&channelId=103230&size=21&sTime=&articleSortType=latest&articleSortCount=0&isIncludeLimitedAskArticle=false&_=1608325605133"
        return url
    def cm_2454_d():
        url_d = "https://www.cmoney.tw/follow/channel/stock-2454?chart=d&type=Personal"
        return url_d
    def cm_2317_u():
        url="https://www.cmoney.tw/follow/channel/getdata/articlelistofstockv2?articleCategory=Personal&channelId=103132&size=21&sTime=&articleSortType=latest&articleSortCount=0&isIncludeLimitedAskArticle=false&_=1608324266782"
        return url
    def cm_2317_d():
        url_d = "https://www.cmoney.tw/follow/channel/stock-2317?chart=d&type=Personal"
        return url_d
    def cm_2330_u():
        url="https://www.cmoney.tw/follow/channel/getdata/articlelistofstockv2?articleCategory=Personal&channelId=103140&size=21&sTime=&articleSortType=latest&articleSortCount=0&isIncludeLimitedAskArticle=false&_=1608321804527"
        return url
    def cm_2330_d():
        url_d = "https://www.cmoney.tw/follow/channel/stock-2330?chart=r&type=Personal"
        return url_d
    def cm_2303_u():
        url="https://www.cmoney.tw/follow/channel/getdata/articlelistofstockv2?articleCategory=Personal&channelId=103123&size=21&sTime=&articleSortType=latest&articleSortCount=0&isIncludeLimitedAskArticle=false&_=1608486533509"
        return url
    def cm_2303_d():
        url_d = "https://www.cmoney.tw/follow/channel/stock-2303?chart=d&type=Personal" 
        return url_d

    def x1(url,url_d):
        js = "var q=document.documentElement.scrollTop=10000"  #selenium 滾輪拉到底設定
        re=requests.post(url)
        data=json.loads(re.text)
        driver = webdriver.Chrome(driver_address) 
        driver.get(url_d)
        #這段主要是為了selenium模擬下拉以後網址會改變，下面這三個是他的起始值和規律
        skipCount=21                                #起始沒有，下拉一次從21開始，第一次以後每次+10
        size=10                                     #起使=21 接下來每次都固定10
        articleId=data[20]["ArtId"]                 # articleId = 每一個Json網頁的最後一人發表的ID
        data_content_list = []
        if today2pm < nowtime < tomorrow12am:       #假設現在時間介於 今天下午兩點與今天半夜12點  代表是今天使用

            #在還沒用selenium往下拉之前只能爬20筆留言的迴圈
            n=0                         #n是計數器功能，用來顯示每一則留言的序號        
            while n<=20:               #第一篇(1~20)爬蟲
                data_content=data[n]["ArtCtn"]
                data_time=data[n]["ArtCteTm"]
                data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                date_time = data_time.replace("/","-")
                date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M') #算出爬出資料的時間，若超過設定的範圍則停止
                n+=1
                if date_time_final > today2pm:
                    data_content_list.append(data_content_final)


            #下面的while是用selenium往下拉之後爬取留言的迴圈  (經觀察，往下拉一次只能爬取9筆留言)
            while date_time_final > today2pm:
                for i in range(0,10):
                    n+=1                    #n是計數器功能，用來顯示每一則留言的序號

                    driver.execute_script(js) #啟動模擬滑鼠滾輪下拉網頁

                    #下拉以後網址會改變，找到規律以後利用規律更改網址來配合下拉
                    url = 'https://www.cmoney.tw/follow/channel/getdata/articlelistmoreofstockv2?articleCategory=Personal&channelId=103681&articleId='+str(articleId)+"&size="+str(size)+"&skipCount="+str(skipCount)+"&articleSortCount=0&sTime=&articleSortType=latest&isIncludeLimitedAskArticle=false&_=1606990418284"
                    articleId=data[9]["ArtId"]
                    skipCount+=10
                    size=10
                    re=requests.post(url)
                    data=json.loads(re.text)
                    data_content=data[i]["ArtCtn"]
                    data_time=data[i]["ArtCteTm"]
                    data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                    date_time = data_time.replace("/","-")
                    date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M') #算出爬出資料的時間，若超過設定的範圍則停止
                    if date_time_final > today2pm:  #判斷爬出的貼完時間是否符合條件
                        data_content_list.append(data_content_final)
            else:
                driver.close()    #不符合條件就關閉chrome模擬器停止爬蟲
        
        #以下邏輯同上        


        elif today12am < nowtime <today2pm:
            n=0
            while n<=20:
                data_content=data[n]["ArtCtn"]
                data_time=data[n]["ArtCteTm"]
                data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                date_time = data_time.replace("/","-")
                date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M')
                n+=1
                if date_time_final > yesterday2pm:
                    data_content_list.append(data_content_final)

            while date_time_final > yesterday2pm:
                for i in range(0,10):
                    n+=1
                    driver.execute_script(js)
                    url = 'https://www.cmoney.tw/follow/channel/getdata/articlelistmoreofstockv2?articleCategory=Personal&channelId=103681&articleId='+str(articleId)+"&size="+str(size)+"&skipCount="+str(skipCount)+"&articleSortCount=0&sTime=&articleSortType=latest&isIncludeLimitedAskArticle=false&_=1606990418284"
                    articleId=data[9]["ArtId"]
                    skipCount+=10
                    size=10
                    re=requests.post(url)
                    data=json.loads(re.text)
                    data_content=data[i]["ArtCtn"]
                    data_time=data[i]["ArtCteTm"]
                    data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                    date_time = data_time.replace("/","-")
                    date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M')
                    if date_time_final > yesterday2pm:
                        data_content_list.append(data_content_final)
            else:
                driver.close()
        return data_content_list
    def content(x1,url,url_d):
        if 0 < nowtime.weekday() < 4 :
            return x1
        elif nowtime.weekday() == 0 and nowtime > today2pm :
            return x1
        elif nowtime.weekday() == 4 and nowtime < today2pm :
            return x1
        else :  #!!!這個else是為了禮拜五六日而設計，因為六日收盤所以禮拜一的大盤並不只有用前一天的新聞做預測。
            js = "var q=document.documentElement.scrollTop=10000"  #selenium 滾輪拉到底設定
            re=requests.post(url)
            data=json.loads(re.text)
            driver = webdriver.Chrome(driver_address) 
            driver.get(url_d)
            #這段主要是為了selenium模擬下拉以後網址會改變，下面這三個是他的起始值和規律
            skipCount=21                                #起始沒有，下拉一次從21開始，第一次以後每次+10
            size=10                                     #起使=21 接下來每次都固定10
            articleId=data[20]["ArtId"]                 # articleId = 每一個Json網頁的最後一人發表的ID
            data_content_list = []
            if nowtime.weekday() == 4 and nowtime > today2pm :         
                    n=0                 
                    while n<=20:
                        data_content=data[n]["ArtCtn"]
                        data_time=data[n]["ArtCteTm"]
                        data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                        date_time = data_time.replace("/","-")
                        date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M') 
                        n+=1
                        if date_time_final > today2pm:
                            data_content_list.append(data_content_final)
                    while date_time_final > today2pm:
                        for i in range(0,10):
                            n+=1 
                            driver.execute_script(js) 
                            url = 'https://www.cmoney.tw/follow/channel/getdata/articlelistmoreofstockv2?articleCategory=Personal&channelId=103681&articleId='+str(articleId)+"&size="+str(size)+"&skipCount="+str(skipCount)+"&articleSortCount=0&sTime=&articleSortType=latest&isIncludeLimitedAskArticle=false&_=1606990418284"
                            articleId=data[9]["ArtId"]
                            skipCount+=10
                            size=10
                            re=requests.post(url)
                            data=json.loads(re.text)
                            data_content=data[i]["ArtCtn"]
                            data_time=data[i]["ArtCteTm"]
                            data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                            date_time = data_time.replace("/","-")
                            date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M')
                            if date_time_final > today2pm:  
                                data_content_list.append(data_content_final)
                    else:
                        driver.close()    

            elif nowtime.weekday() == 0 and nowtime < today2pm :
                if today12am < nowtime <today2pm:
                    n=0
                    while n<=20:
                        data_content=data[n]["ArtCtn"]
                        data_time=data[n]["ArtCteTm"]
                        data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                        date_time = data_time.replace("/","-")
                        date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M')
                        n+=1
                        if date_time_final > today2pm-datetime.timedelta(days = 3):
                            data_content_list.append(data_content_final)                    
                    while date_time_final > today2pm-datetime.timedelta(days = 3):
                        for i in range(0,10):
                            n+=1
                            driver.execute_script(js)
                            url = 'https://www.cmoney.tw/follow/channel/getdata/articlelistmoreofstockv2?articleCategory=Personal&channelId=103681&articleId='+str(articleId)+"&size="+str(size)+"&skipCount="+str(skipCount)+"&articleSortCount=0&sTime=&articleSortType=latest&isIncludeLimitedAskArticle=false&_=1606990418284"
                            articleId=data[9]["ArtId"]
                            skipCount+=10
                            size=10
                            re=requests.post(url)
                            data=json.loads(re.text)
                            data_content=data[i]["ArtCtn"]
                            data_time=data[i]["ArtCteTm"]
                            data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                            date_time = data_time.replace("/","-")
                            date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M')
                            if date_time_final > today2pm-datetime.timedelta(days = 3) :
                                data_content_list.append(data_content_final)
                    else:
                        driver.close()
            elif nowtime.weekday() == 5 :
                    n=0
                    while n<=20:
                        data_content=data[n]["ArtCtn"]
                        data_time=data[n]["ArtCteTm"]
                        data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                        date_time = data_time.replace("/","-")
                        date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M')
                        n+=1
                        if date_time_final > yesterday2pm:
                            data_content_list.append(data_content_final)

                    while date_time_final > yesterday2pm :
                        for i in range(0,10):
                            n+=1
                            driver.execute_script(js)
                            url = 'https://www.cmoney.tw/follow/channel/getdata/articlelistmoreofstockv2?articleCategory=Personal&channelId=103681&articleId='+str(articleId)+"&size="+str(size)+"&skipCount="+str(skipCount)+"&articleSortCount=0&sTime=&articleSortType=latest&isIncludeLimitedAskArticle=false&_=1606990418284"
                            articleId=data[9]["ArtId"]
                            skipCount+=10
                            size=10
                            re=requests.post(url)
                            data=json.loads(re.text)
                            data_content=data[i]["ArtCtn"]
                            data_time=data[i]["ArtCteTm"]
                            data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                            date_time = data_time.replace("/","-")
                            date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M')
                            if date_time_final > yesterday2pm :
                                data_content_list.append(data_content_final)
                    else:
                        driver.close()

            elif nowtime.weekday() == 6 :
                    n=0
                    while n<=20:
                        data_content=data[n]["ArtCtn"]
                        data_time=data[n]["ArtCteTm"]
                        data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                        date_time = data_time.replace("/","-")
                        date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M')
                        n+=1
                        if date_time_final > yesterday2pm-datetime.timedelta(days = 1) :
                            data_content_list.append(data_content_final)

                    while date_time_final > yesterday2pm-datetime.timedelta(days = 1) :
                        for i in range(0,10):
                            n+=1
                            driver.execute_script(js)
                            url = 'https://www.cmoney.tw/follow/channel/getdata/articlelistmoreofstockv2?articleCategory=Personal&channelId=103681&articleId='+str(articleId)+"&size="+str(size)+"&skipCount="+str(skipCount)+"&articleSortCount=0&sTime=&articleSortType=latest&isIncludeLimitedAskArticle=false&_=1606990418284"
                            articleId=data[9]["ArtId"]
                            skipCount+=10
                            size=10
                            re=requests.post(url)
                            data=json.loads(re.text)
                            data_content=data[i]["ArtCtn"]
                            data_time=data[i]["ArtCteTm"]
                            data_content_final = data_content[data_content.index("<div class='main-content'>")+26:data_content.index("</div>")].replace("<br>","")
                            date_time = data_time.replace("/","-")
                            date_time_final = datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M')
                            if date_time_final > yesterday2pm-datetime.timedelta(days = 1) :
                                data_content_list.append(data_content_final)
                    else:
                        driver.close()
            else:
                driver.close()
        return data_content_list
    #會開一堆模擬器一次把全部跑完，看下面執行結果就懂。
    gossip_TWA00 = content(x1(cm_TWA00_u(),cm_TWA00_d()),cm_TWA00_u(),cm_TWA00_d())
    gossip_2330 = content(x1(cm_2330_u(),cm_2330_d()),cm_2330_u(),cm_2330_d())
    gossip_2317 = content(x1(cm_2317_u(),cm_2317_d()),cm_2317_u(),cm_2317_d())
    gossip_2454 = content(x1(cm_2454_u(),cm_2454_d()),cm_2454_u(),cm_2454_d())
    gossip_2303 = content(x1(cm_2303_u(),cm_2303_d()),cm_2303_u(),cm_2303_d())

    if gossip_TWA00 == []:
        gossip_TWA00 = ["上漲", "下跌"]
    if gossip_2330 == []:
        gossip_2330 = ["上漲", "下跌"]
    if gossip_2317 == []:
        gossip_2317 = ["上漲", "下跌"]
    if gossip_2454 == []:
        gossip_2454 = ["上漲", "下跌"]
    if gossip_2303 == []:
        gossip_2303 = ["上漲", "下跌"]
        
    list_2330 = yahoo("2330")+eco("2330")
    list_2317 = yahoo("2317")+eco("2317")
    list_2454 = yahoo("2454")+eco("2454")
    list_2303 = yahoo("2303")+eco("2303")
    TW001_news = cn()+eco("大盤")



    #新聞標題
    ########2330##########

    a = pd.DataFrame(list_2330, columns=['text'])

    def 去特殊符號(t):
        # 特殊符號可以自己增補
        t = re.sub(r"[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",t)
        t = re.sub(r'[-，+、@,？⊂・ω{}●►❖★！!：\\/(=)…（）『』%《》$;；」:?=<>"／.&#"「_【】]', "",t)    
        # 去除數字
        t = re.sub(r"[0123456789]","",t)    
        # 去除英文  
        t = re.sub(r"[a-zA-Z]","",t)      
        return t   
                
    documents_l = open(path+"documents.pickle","rb")
    documents = pickle.load(documents_l)
    documents_l.close()
        

    def ngrams(input, min, max):
        input = input.split(' ')
        output = []
        texts=''  
        for c in range(min,max+1):
            n=c
            for i in range(len(input)-n+1):
                texts=' '.join(input[i:i+n])
                output.append(texts)
        return output

    train_set = []
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for (words, sentiment) in documents :
        # 所有的字詞 與 情緒標記
        word_list=ngrams(words, 1,1) 
        
        # 去除停用字 stopwords
        word_list=[e for e  in word_list if e not in stopwords] 
        
    
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))


    # 取得文本中所有的字詞 list，有重覆
    def get_words_in_train(train_set):
        all_words = []
        for (words, sentiment) in train_set:
            all_words.extend(words)
        return all_words
    # 取得字詞列表，不重復 dictionary_key
    def get_word_features(wordlist):
        wordlist = nltk.FreqDist(wordlist)
        word_features = wordlist.keys()
        return word_features
    # 字詞是否(True or False)在文件有被引用，dictionary
    def extract_features(train_set):
        document_words = set(train_set)
        features = {}
        for word in word_features:
            features['contains(%s)' % word] = (word in document_words)
        return features

    # 字詞列表， dictionary_key 不重復
    word_features = get_word_features(get_words_in_train(train_set))

    classifier_f = open(path+"naivebayes_senti.pickle", "rb")
    classifier = pickle.load(classifier_f)
    classifier_f.close()

    # 中文斷字，並刪除無意義的符號
    # 設定斷詞字典
    jieba.set_dictionary(path+"dict_all.txt")    
    
    # 增加新詞，字典可以隨時叫出修改， 增添新的詞彙。 
    # jieba.load_userdict(path+ "userdict.txt")   

    seg_list_2330_news=[]
    for t in a.text:
        t = 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)

        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '

        # string 轉為 list
        seg_list_2330_news.append(seg_string)   
        
    a['token']=seg_list_2330_news

    token = a.token
    token[0].split()

    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
        
    pred_2330_news = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred_2330_news.append(prediction)

    a["pred"] = pred_2330_news

    news_2330 = int(len(a[a.pred=="positive"]))
    news_2330_n = int(len(a[a.pred=="negative"]))



    ########2317##########

    b = pd.DataFrame(list_2317, columns=['text'])

    seg_list_2317_news=[]
    for t in b.text:
        t = 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)

        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '

        # string 轉為 list
        seg_list_2317_news.append(seg_string)   
        
    b['token']=seg_list_2317_news

    token = b.token
    token[0].split()

    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:    
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
        
    pred_2317_news = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred_2317_news.append(prediction)

    b["pred"] = pred_2317_news

    news_2317 = int(len(b[b.pred=="positive"]))
    news_2317_n = int(len(b[b.pred=="negative"]))


    ########2454##########

    c = pd.DataFrame(list_2454, columns=['text'])

    seg_list_2454_news=[]
    for t in c.text:
        t = 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)

        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '

        # string 轉為 list
        seg_list_2454_news.append(seg_string)   
        
    c['token']=seg_list_2454_news

    token = c.token
    token[0].split()

    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
        
    pred_2454_news = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred_2454_news.append(prediction)

    c["pred"] = pred_2454_news

    news_2454 = int(len(c[c.pred=="positive"]))
    news_2454_n = int(len(c[c.pred=="negative"]))
    



    ########2303##########

    d = pd.DataFrame(list_2303, columns=['text'])

    seg_list_2303_news=[]
    for t in d.text:
        t = 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)

        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '

        # string 轉為 list
        seg_list_2303_news.append(seg_string)   
        
    d['token']=seg_list_2303_news

    token = d.token
    token[0].split()

    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:    
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
        
    pred_2303_news = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred_2303_news.append(prediction)

    d["pred"] = pred_2303_news

    news_2303 = int(len(d[d.pred=="positive"]))
    news_2303_n = int(len(d[d.pred=="negative"]))
    #######大盤#############

    j = pd.DataFrame(TW001_news, columns=['text'])

    seg_list_TW001_news=[]
    for t in j.text:
        t = 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)

        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '

        # string 轉為 list
        seg_list_TW001_news.append(seg_string)   
        
    j['token']=seg_list_TW001_news

    token = j.token
    token[0].split()

    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:    
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
        
    pred_TW001_news = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred_TW001_news.append(prediction)

    j["pred"] = pred_TW001_news

    TW001_news = int(len(j[j.pred=="positive"]))
    TW001_news_n = int(len(j[j.pred=="negative"]))


    #留言
    #######2330#####################


    f = pd.DataFrame(gossip_2330, columns=['text'])

    documents_l = open(path+"documents_comments.pickle","rb")
    documents_comments = pickle.load(documents_l)
    documents_l.close()

    train_set = []
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for (words, sentiment) in documents_comments :
        # 所有的字詞 與 情緒標記
        word_list=ngrams(words, 1,1) 
        
        # 去除停用字 stopwords
        word_list=[e for e  in word_list if e not in stopwords] 
        
    
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))

    # 字詞列表， dictionary_key 不重復
    word_features = get_word_features(get_words_in_train(train_set))

    classifier_f = open(path+"naivebayes_senti_comments.pickle", "rb")
    classifier = pickle.load(classifier_f)
    classifier_f.close()

    seg_list_2330=[]
    for t in f.text:
        t = 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)

        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '

        # string 轉為 list
        seg_list_2330.append(seg_string)   
        
    f['token']=seg_list_2330

    token = f.token
    token[0].split()

    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:
        # 所有的字詞 與 情緒標記
        word_list=ngrams(words, 1,1) 
        
        # 去除停用字 stopwords
        word_list=[e for e  in word_list if e not in stopwords] 
        
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
        
    pred_2330 = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred_2330.append(prediction)

    f["pred"] = pred_2330

    f_2330 = int(len(f[f.pred=="positive"]))
    f_2330_n = int(len(f[f.pred=="negative"]))

    #######2317#####################


    g = pd.DataFrame(gossip_2317, columns=['text'])

    seg_list_2317=[]
    for t in g.text:
        t = 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)

        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '

        # string 轉為 list
        seg_list_2317.append(seg_string)   
        
    g['token']=seg_list_2317

    token = g.token
    token[0].split()

    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:
        # 所有的字詞 與 情緒標記
        word_list=ngrams(words, 1,1) 
        
        # 去除停用字 stopwords
        word_list=[e for e  in word_list if e not in stopwords] 
        
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
        
    pred_2317 = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred_2317.append(prediction)

    g["pred"] = pred_2317

    g_2317 = int(len(g[g.pred=="positive"]))
    g_2317_n = int(len(g[g.pred=="negative"]))


    #######2454#####################


    h = pd.DataFrame(gossip_2454, columns=['text'])

    seg_list_2454=[]
    for t in h.text:
        t = 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)

        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '

        # string 轉為 list
        seg_list_2454.append(seg_string)   
        
    h['token']=seg_list_2454

    token = h.token
    token[0].split()

    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:
        # 所有的字詞 與 情緒標記
        word_list=ngrams(words, 1,1) 
        
        # 去除停用字 stopwords
        word_list=[e for e  in word_list if e not in stopwords] 
        
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
        
    pred_2454 = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred_2454.append(prediction)

    h["pred"] = pred_2454

    h_2454 = int(len(h[h.pred=="positive"]))
    h_2454_n = int(len(h[h.pred=="negative"]))



    #######2303#####################


    i = pd.DataFrame(gossip_2303, columns=['text'])

    seg_list_2303=[]
    for t in i.text:
        t = 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)

        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '

        # string 轉為 list
        seg_list_2303.append(seg_string)   
        
    i['token']=seg_list_2303

    token = i.token
    token[0].split()

    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:
        # 所有的字詞 與 情緒標記
        word_list=ngrams(words, 1,1) 
        
        # 去除停用字 stopwords
        word_list=[e for e  in word_list if e not in stopwords] 
        
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
        
    pred_2303 = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred_2303.append(prediction)

    i["pred"] = pred_2303

    i_2303 = int(len(i[i.pred=="positive"]))
    i_2303_n = int(len(i[i.pred=="negative"]))

    #######大盤###############
    k = pd.DataFrame(gossip_TWA00, columns=['text'])

    seg_list_TWA00=[]
    for t in k.text:
        t = 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)

        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '

        # string 轉為 list
        seg_list_TWA00.append(seg_string)   
        
    k['token']=seg_list_TWA00

    token = k.token
    token[0].split()

    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:
        # 所有的字詞 與 情緒標記
        word_list=ngrams(words, 1,1) 
        
        # 去除停用字 stopwords
        word_list=[e for e  in word_list if e not in stopwords] 
        
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))

    pred_TWA00 = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred_TWA00.append(prediction)

    k["pred"] = pred_TWA00
    
    k_TWA00 = int(len(k[k.pred=="positive"]))
    k_TWA00_n = int(len(k[k.pred=="negative"]))
    return news_2330,news_2330_n,news_2317,news_2317_n,news_2454,news_2454_n,news_2303,news_2303_n,TW001_news,TW001_news_n,f_2330,f_2330_n,g_2317,g_2317_n,h_2454,h_2454_n,i_2303,i_2303_n,k_TWA00,k_TWA00_n


In [3]:

TAIEX_result = run_TAIEX()

# news_2330 =     #a
# news_2330_n   #a
# news_2317   #b
# news_2317_n   #b
# news_2454   #c
# news_2454_n   #c
# news_2303   #d
# news_2303_n   #d
# TW001_news   #j
# TW001_news_n   #j
# f_2330
# f_2330_n
# g_2317
# g_2317_n
# h_2454
# h_2454_n
# i_2303
# i_2303_n
# k_TWA00
# k_TWA00_n

Building prefix dict from C:\My python\dict_all.txt ...
Dumping model to file cache C:\Users\apz33\AppData\Local\Temp\jieba.u00c367b986563c0985937721e33bcd72.cache
Loading model cost 0.052014827728271484 seconds.
Prefix dict has been built succesfully.


In [3]:
news_2330 = TAIEX_result[0]   #a
news_2330_n = TAIEX_result[1]   #a
news_2317 = TAIEX_result[2]   #b
news_2317_n = TAIEX_result[3]   #b
news_2454 = TAIEX_result[4]   #c
news_2454_n = TAIEX_result[5]   #c
news_2303 = TAIEX_result[6]   #d
news_2303_n = TAIEX_result[7]   #d
TW001_news = TAIEX_result[8]   #j
TW001_news_n = TAIEX_result[9]   #j
f_2330 = TAIEX_result[10]
f_2330_n = TAIEX_result[11]
g_2317 = TAIEX_result[12]
g_2317_n = TAIEX_result[13]
h_2454 = TAIEX_result[14]
h_2454_n = TAIEX_result[15]
i_2303 = TAIEX_result[16]
i_2303_n = TAIEX_result[17]
k_TWA00 = TAIEX_result[18]
k_TWA00_n = TAIEX_result[19]


In [4]:
total_comments1 = round(int(k_TWA00) / int(k_TWA00 + k_TWA00_n)*100, 2)
total_comments1_minus = 100-total_comments1
    # print("大盤留言看跌比率", 100-total_comments1)

TWA00_title = round(int(TW001_news) / int(TW001_news + TW001_news_n)*100, 2)
# print("大盤新聞看漲比率:", TWA00_title)
TWA00_title_minus = 100-TWA00_title
    

total_comments = round(int(f_2330) / (int(f_2330) + int(f_2330_n))*100*0.8185201258543994 + int(g_2317) / (int(g_2317) + int(g_2317_n))*100*0.0749511771726158 + int(h_2454) / (int(h_2454) + int(h_2454_n))*100*0.073478355213193 + int(i_2303) / (int(i_2303) + int(i_2303_n))*100*0.0330503417597917, 2)
total_title = round(int(news_2330) / (int(news_2330) + int(news_2330_n))*100*0.8185201258543994 + int(news_2317 ) / (int(news_2317) + int(news_2317_n))*100*0.0749511771726158 + int(news_2454) / (int(news_2454) + int(news_2454_n))*100*0.073478355213193 + int(news_2303) / (int(news_2303) + int(news_2303_n))*100*0.0330503417597917, 2)
# print("權值股留言看漲比率", total_comments)
    


# print("權值新聞與留言加權看漲比率", total)





In [5]:
def plot_TAIEX(x):
    from matplotlib import rcParams # 中文設定1: 楷體字
    rcParams["font.sans-serif"] = ["kaiu"] 
    rcParams["font.family"] ="DFKai-sb"
    matplotlib.use('TkAgg')
    matplotlib.rcParams['font.sans-serif'] = ['SimHei'] #组件字体
    total1 = round(total_comments * float(x) + total_title * (1-float(x)), 2)
    total1
    total_list1= [total1]
    total_list_n1= [100-total1]
    data_total1 = list(zip(total_list1,total_list_n1))
    df_total1 = pd.DataFrame(data_total1, columns=['看漲','看跌'])
    names = ['看漲','看跌']
    # 轉為 dictionary
    data_total1=dict(df_total1[['看漲','看跌']].transpose())
    labels_total1 = list(data_total1.keys())
    data_total1 = np.array(list(data_total1.values()))   
    # 各筆資料總合，因為點計算百分比
    data_total_cum1 = data_total1.cumsum(axis=1)
    category_total_colors1 = plt.get_cmap('RdYlGn')(np.linspace(0.15, 0.9, data_total1.shape[1]))
    fig, ax = plt.subplots(figsize=(20, 1)) # 圖區大小(寬，高)
    ax.set_yticks(range(len(labels_total1)))
    ax.set_yticklabels(range(len(labels_total1)))# y 刻度標籤
    # y 由小到大，方向調整
    ax.invert_yaxis()
    # x 不出現刻度
    ax.xaxis.set_visible(False)
    # 設定 x 範圍，讓b1,b2,b3 加總後，都一樣大
    ax.set_xlim(0, np.sum(data_total1, axis=1).max())
    for i, (name, color) in enumerate(zip(names, category_total_colors1)):
        widths_total1 = data_total1[:, i]
        starts_total1 = data_total_cum1[:, i] - widths_total1
        ax.barh(labels_total1,  widths_total1, left=starts_total1, height=1,label=name, color=color)
        xcenters = starts_total1 + widths_total1 / 2
        # 讀取顏色資訊
        r, g, b, _ = color
        # 決定文字顏色，有兩種，白色與灰色
        text_color = 'white' if r * g * b < 0.5 else 'black'
        # 加入文字
        for y, (x, c) in enumerate(zip(xcenters, widths_total1)):
            ax.text(x, y, str(int(c))+"%", ha='center', va='center',color=text_color, fontsize=18)#圖形內文字設定
    # 增加圖例     
    plt.title('權值股新聞與留言加權漲跌比率',loc='center',fontsize=20,)
    ax.legend(ncol=len(names), bbox_to_anchor=(0, 1), loc='lower left', fontsize=10) #bbox_to_anchor圖例高度
    plt.tight_layout() 
    plt.show()

   


    
    
    
   

In [6]:
def plot_TAIEX_1(x):
    #合併留言與新聞預測權值股 圖6
    total = round(total_comments1 * float(x) + TWA00_title * (1-float(x)), 2)



    total_list= [total]
    total_list_n= [100-total]
    data_total = list(zip(total_list,total_list_n))
    df_total = pd.DataFrame(data_total, columns=['看漲','看跌'])


    names = ['看漲','看跌']
    # 轉為 dictionary
    data_total=dict(df_total[['看漲','看跌']].transpose())

    labels_total = list(data_total.keys())

    data_total = np.array(list(data_total.values()))   
    # 各筆資料總合，因為點計算百分比
    data_total_cum = data_total.cumsum(axis=1)
    category_total_colors = plt.get_cmap('RdYlGn')(np.linspace(0.15, 0.9, data_total.shape[1]))

    fig, ax = plt.subplots(figsize=(20, 1)) # 圖區大小(寬，高)
    ax.set_yticks(range(len(labels_total)))
    ax.set_yticklabels(range(len(labels_total)))# y 刻度標籤
    # y 由小到大，方向調整
    ax.invert_yaxis()
    # x 不出現刻度
    ax.xaxis.set_visible(False)
    # 設定 x 範圍，讓b1,b2,b3 加總後，都一樣大
    ax.set_xlim(0, np.sum(data_total, axis=1).max())

    for i, (name, color) in enumerate(zip(names, category_total_colors)):
        widths_total = data_total[:, i]
        starts_total = data_total_cum[:, i] - widths_total
        ax.barh(labels_total,  widths_total, left=starts_total, height=1,label=name, color=color)
        xcenters = starts_total + widths_total / 2
        # 讀取顏色資訊
        r, g, b, _ = color
        # 決定文字顏色，有兩種，白色與灰色
        text_color = 'white' if r * g * b < 0.5 else 'black'
        # 加入文字
        for y, (x, c) in enumerate(zip(xcenters, widths_total)):
            ax.text(x, y, str(int(c))+"%", ha='center', va='center',color=text_color, fontsize=18)#圖形內文字設定
    # 增加圖例     
    plt.title('大盤新聞與留言加權漲跌比率',loc='center',fontsize=20,)
    ax.legend(ncol=len(names), bbox_to_anchor=(0, 1), loc='lower left', fontsize=10) #bbox_to_anchor圖例高度
    plt.tight_layout()
    plt.show()
    


In [7]:

def customized1(): 
    TWA00_title_list = [TWA00_title]
    TWA00_title_list_n = [100-TWA00_title]
    data_TWA00_title = list(zip(TWA00_title_list,TWA00_title_list_n))
    df_TWA00_title = pd.DataFrame(data_TWA00_title, columns=['看漲','看跌']) 
    names = ['看漲','看跌']
    # 轉為 dictionary
    data_TWA00_title=dict(df_TWA00_title[['看漲','看跌']].transpose())
    # dictionary 取 key 為 labels，即 0 到 2
    labels_TWA00_title = list(data_TWA00_title.keys())
    data_TWA00_title = np.array(list(data_TWA00_title.values()))
    # 各筆資料總合，因為點計算百分比
    data_TWA00_title_cum = data_TWA00_title.cumsum(axis=1)
    category_TWA00_title_colors = plt.get_cmap('RdYlGn')(np.linspace(0.15, 0.9, data_TWA00_title.shape[1]))

    fig, ax = plt.subplots(figsize=(20, 1)) # 圖區大小(寬，高)

    ax.set_yticks(range(len(labels_TWA00_title)))

    ax.set_yticklabels(range(len(labels_TWA00_title)))
    # y 由小到大，方向調整
    ax.invert_yaxis()
    # x 不出現刻度
    ax.xaxis.set_visible(False)
    # 設定 x 範圍，讓b1,b2,b3 加總後，都一樣大
    ax.set_xlim(0, np.sum(data_TWA00_title, axis=1).max())
    for i, (name, color) in enumerate(zip(names,category_TWA00_title_colors)):
        # 寬度，即變數值
        widths_TWA00_title = data_TWA00_title[:, i]
        starts_TWA00_title = data_TWA00_title_cum[:, i] - widths_TWA00_title
        ax.barh(labels_TWA00_title,  widths_TWA00_title, left=starts_TWA00_title, height=1,label=name, color=color)
        xcenters = starts_TWA00_title + widths_TWA00_title /2
        # 讀取顏色資訊
        r, g, b, _ = color
        # 決定文字顏色，有兩種，白色與灰色
        text_color = 'white' if r * g * b < 0.5 else 'black'
        # 加入文字
        for y, (x, c) in enumerate(zip(xcenters, widths_TWA00_title)):
            ax.text(x, y, str(int(c))+"%", ha='center', va='center',color=text_color, fontsize=18)#圖形內文字設定
        # 增加圖例     
    plt.title('大盤新聞風向漲跌比率',loc='center',fontsize=20,)
    ax.legend(ncol=len(names), bbox_to_anchor=(0, 1), loc='lower left', fontsize=10) #bbox_to_anchor圖例高度
    plt.tight_layout()
    plt.show


In [8]:
def openwin():
    roots = tk.Tk() 
    roots.geometry("650x300") 
    roots.title("個股預測結果")
    r_label_1 = tk.Label(roots , text = s)
    r_label_1.pack()
    roots.mainloop()
def ind_stock_predictor(x):

    def yahoo(x): #Yahoo爬蟲===>yahoo沒有大盤新聞專頁、因此只能用權值股預測大盤，x輸入權值股即可，另外yahoo可以應用於個股查詢功能。
        yahoo_title_list= []
        yahoo_time_list = []
        yahoo_address_list = []
        final_list = []
    
        for w in range(1,3): 
            url_yahoo = "https://tw.stock.yahoo.com/q/h?s="+x+"&pg="+ str(w) #str(s)代表搜尋欄位 str(w)代表頁數
            re = requests.get(url = url_yahoo)
            soup_address = BeautifulSoup(re.text,"html.parser") 
            yahoo_address = soup_address.find_all("td",{"height":"37"})
    
            for i in yahoo_address:
                text = i.text
                if text[0] =="《":
                    yahoo_title_list.append(text[text.index("》")+1:])
                else:            
                    yahoo_title_list.append(i.text)
                
            for i in yahoo_address:
                k = "https://tw.stock.yahoo.com/"+i.a["href"]
                yahoo_address_list.append(k) #第w頁的所有標題網址
    
        for x in yahoo_address_list:  #第w頁第i個標題網址
            re = requests.get(url =x)
            soup_content = BeautifulSoup(re.text,"html.parser")
            yahoo_time = soup_content.find("time",{"class":"caas-attr-meta-time"})  #上傳時間
            yahoo_time = str(yahoo_time["datetime"]).replace("T", " ").replace(".000Z", "")
            if yahoo_time == "":
                continue
            yahoo_time = datetime.datetime.strptime(yahoo_time, "%Y-%m-%d %H:%M:%S")+datetime.timedelta(hours = 8)
            yahoo_time_list.append(yahoo_time)
    
        for n,t in zip(yahoo_title_list,yahoo_time_list):
            if 0 < nowtime.weekday() < 4 :
                if today2pm < nowtime < tomorrow12am and t > today2pm :
                    final_list.append(n)
                elif today12am < nowtime < today2pm and t > yesterday2pm :
                    final_list.append(n)
                else :
                    continue
            elif nowtime.weekday() == 0 and nowtime > today2pm :
                if today2pm < nowtime < tomorrow12am and t > today2pm :
                    final_list.append(n)
                elif today12am < nowtime < today2pm and t > yesterday2pm :
                    final_list.append(n)
                else :
                    continue
            elif nowtime.weekday() == 4 and nowtime < today2pm :
                if today2pm < nowtime < tomorrow12am and t > today2pm :
                    final_list.append(n)
                elif today12am < nowtime < today2pm and t > yesterday2pm :
                    final_list.append(n)
                else :
                    continue
            else :
                if nowtime.weekday() == 4 and nowtime > today2pm :
                    if t > today2pm :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 0 and nowtime < today2pm :
                    if t > today2pm-datetime.timedelta(days = 3) :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 5 :
                    if t > yesterday2pm :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 6 :
                    if t > yesterday2pm-datetime.timedelta(days = 1) :
                        final_list.append(n)
                    else :
                        continue
                else :
                        continue
        if final_list == [] :
            final_list.append("上漲")
            final_list.append("下跌")
        return final_list 
    
    
    def eco(x): #經濟日報不論要應用在大盤預測或是個股搜索都可以，若要預測大盤，則x輸入"大盤" 和其他權值股。
        eco_address_list = []
        eco_time_list = []
        eco_title_list = []
        for w in range(1,3): #先爬出搜尋2330的所有新聞網址 (先假設爬五頁)
            url_eco = "https://money.udn.com/search/result/1001/"+x+"/"+str(w)
    
            headers = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
                    "cookie":'_ga=GA1.2.927708406.1601618986; _fbp=fb.1.1601618985824.706528271; _ss_pp_id=b3c8335fbeb19f9adaf1601590188761; dable_uid=53822174.1601061552644; __dtsu=4C30160291899678DF25A79BB792ADBD; truvid_protected={"val":"c","level":2,"geo":"TW","timestamp":1605071289}; __auc=04d5a0b6175f83e7d1a75f55ca6; __erEvntUid=noset; __eruid=d8400cdf-4c38-c641-a4d9-87f2d35c2f8a; _fbc=fb.1.1606187751623.IwAR32rqYXY1HwRAL_qIat0HFknrbrsfzraHTaRIhOYM6tqU0r5y3hnMNNWi4; __BWfp=c1606187759520x1a75e6b90; _gid=GA1.2.630643660.1607095760; __gads=ID=56b9d1fd37dbdf4b:T=1607095770:S=ALNI_MZoFn8UvghJvOmXcQR4TUlpiIJPOg; _gliaplayer_user_info={%22city%22:%22shibuya%20city%22%2C%22uid%22:%22a96b4d50-0182-11eb-9ff0-0fb12ecbca9d%22%2C%22country%22:%22TW%22%2C%22region%22:%2213%22%2C%22source%22:%22CF%22%2C%22latlong%22:%2235.661971%2C139.703795%22%2C%22ip%22:%2261.58.108.239%22}; meter_expires=Fri, 11 Dec 2020 17:55:35 GMT; ucfunnel_uid=ad86a035-a317-3d62-8366-fb6443c6f307; __retuid=1a903bd2-775a-d99b-c37a-65052f36a210; csrf_cookie_name=afd2db682c0216bd0166ddd5f541e09b; __asc=2d7132f217631ad0c388694c5f4; gliaplayer_ssid=8c31cab0-36c6-11eb-8837-29e472ee0491; meter_articleIds={"5066965":1,"5068548":1}; meter_day_read_pages={"20201205":2}; meter_limit_day7=2; meter_limit_day1=2; meter_1=2; meter_7=2; _td=f7597bf8-e560-44e6-b888-c97d831c391a; __retfs=fSes-866ff7c9-b611-95f-11d2; __lastv=0'}
    
            re = requests.get(url = url_eco, headers = headers )
    

            soup_address = BeautifulSoup(re.text,"html.parser") 
        #     eco_address = soup_address.find_all(href=r.compile("https://money.udn.com/money/story"))
            eco_title = soup_address.find("div",id="search_content")
            eco_title = str(eco_title)
            soup_title = BeautifulSoup(eco_title,"html.parser")
            eco_address = soup_title.find_all("a")
            eco_title1 = soup_title.find_all("h3")
    
            for n in eco_title1:
                eco_title_list.append(n.text)
            for i in eco_address:
                eco_address_list.append(i["href"])
    
        for x in eco_address_list:  #根據列表中的網址爬取標題
            re = requests.get(url = x, headers = headers )
            soup_time = BeautifulSoup(re.text,"html.parser")
            soup_time1 = soup_time.find("div",{"class":"shareBar__info"})
            if soup_time1 == "":
                continue
            soup_time2 = soup_time1.text[:17].replace("\n","")
    
            soup_time3 = datetime.datetime.strptime(soup_time2,'%Y-%m-%d %H:%M')
            eco_time_list.append(soup_time3)
    
    
        final_list = []
    
        for n,t in zip(eco_title_list,eco_time_list):
                if 0 < nowtime.weekday() < 4 :
                    if today2pm < nowtime < tomorrow12am and t > today2pm :
                        final_list.append(n)
                    elif today12am < nowtime < today2pm and t > yesterday2pm :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 0 and nowtime > today2pm :
                    if today2pm < nowtime < tomorrow12am and t > today2pm :
                        final_list.append(n)
                    elif today12am < nowtime < today2pm and t > yesterday2pm :
                        final_list.append(n)
                    else :
                        continue
                elif nowtime.weekday() == 4 and nowtime < today2pm :
                    if today2pm < nowtime < tomorrow12am and t > today2pm :
                        final_list.append(n)
                    elif today12am < nowtime < today2pm and t > yesterday2pm :
                        final_list.append(n)
                    else :
                        continue
                else :
                    if nowtime.weekday() == 4 and nowtime > today2pm :
                        if t > today2pm :
                            final_list.append(n)
                        else :
                            continue
                    elif nowtime.weekday() == 0 and nowtime < today2pm :
                        if t > today2pm-datetime.timedelta(days = 3) :
                            final_list.append(n)
                        else :
                            continue
                    elif nowtime.weekday() == 5 :
                        if t > yesterday2pm :
                            final_list.append(n)
                        else :
                            continue
                    elif nowtime.weekday() == 6 :
                        if t > yesterday2pm-datetime.timedelta(days = 1) :
                            final_list.append(n)
                        else :
                            continue
                    else :
                            continue
        if final_list == [] :
            final_list.append("上漲")
            final_list.append("下跌")
        return final_list

    individual_stock = eco(x)+yahoo(x)   #經濟日報+雅虎的個股新聞   目前問題是有點少
    
    a = pd.DataFrame(individual_stock, columns=['text'])
    
    def 去特殊符號(t):
        import re
        # 特殊符號可以自己增補
        t = re.sub(r"[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",t)
        t = re.sub(r'[-，+、@,？⊂・ω{}●►❖★！!：\\/(=)…（）『』%《》$;；」:?=<>"／.&#"「_【】]', "",t)    
        # 去除數字
        t = re.sub(r"[0123456789]","",t)    
        # 去除英文  
        t = re.sub(r"[a-zA-Z]","",t)      
        return t  
    
    # 中文斷字，並刪除無意義的符號
    
                
    documents_l = open(path+"documents.pickle","rb")
    documents = pickle.load(documents_l)
    documents_l.close()
        
    
    def ngrams(input, min, max):
        input = input.split(' ')
        output = []
        texts=''  
        for c in range(min,max+1):
            n=c
            for i in range(len(input)-n+1):
                texts=' '.join(input[i:i+n])
                output.append(texts)
        return output
    
    train_set = []
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for (words, sentiment) in documents :
        # 所有的字詞 與 情緒標記
        word_list=ngrams(words, 1,1) 
        
        # 去除停用字 stopwords
        word_list=[e for e  in word_list if e not in stopwords] 
        
    
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
    
    
    # 取得文本中所有的字詞 list，有重覆
    def get_words_in_train(train_set):
        all_words = []
        for (words, sentiment) in train_set:
            all_words.extend(words)
        return all_words
    # 取得字詞列表，不重復 dictionary_key
    def get_word_features(wordlist):
        wordlist = nltk.FreqDist(wordlist)
        word_features = wordlist.keys()
        return word_features
    # 字詞是否(True or False)在文件有被引用，dictionary
    def extract_features(train_set):
        document_words = set(train_set)
        features = {}
        for word in word_features:
            features['contains(%s)' % word] = (word in document_words)
        return features
    
    # 字詞列表， dictionary_key 不重復
    word_features = get_word_features(get_words_in_train(train_set))
    
    classifier_f = open(path+"naivebayes_senti.pickle", "rb")
    classifier = pickle.load(classifier_f)
    classifier_f.close()
    
    # 中文斷字，並刪除無意義的符號
    # 設定斷詞字典
    jieba.set_dictionary(path+"dict_all.txt")    
    
    # 增加新詞，字典可以隨時叫出修改， 增添新的詞彙。 
    # jieba.load_userdict(path+ "userdict.txt")   
    
    seg_list=[]
    for t in a.text:
        t= 去特殊符號(t)
        # 結巴分詞，引數為 string。
        result = jieba.cut(t)
    
        # 讀取中文斷詞結果，存為 string，中間以空白隔開
        seg_string=''
        for word in result:
            seg_string= seg_string + word+' '
    
        # string 轉為 list
        seg_list.append(seg_string)   
        
    a['token']=seg_list
    
    token = a.token
    token[0].split()
    
    test_set = list(token)
    # 把測試組的所有資料叫進來，含字詞及情緒標記 #######
    for words in test_set:
        # 所有的字詞 與 情緒標記
        word_list=ngrams(words, 1,1) 
        
        # 去除停用字 stopwords
        word_list=[e for e  in word_list if e not in stopwords] 
        
        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        train_set.append((words_filtered, sentiment))
        
    yes = 0
    pred = []
    for t in range(0,len(token)):
        prediction=classifier.classify(extract_features(token[t].split()))
        pred.append(prediction)
    
    a["pred"] = pred
    total_k = round(int(len(a[a.pred=="positive"])) / (int(len(a[a.pred=="positive"])) + int(len(a[a.pred=="negative"])))*100, 2)
    
    # 回傳區
    # print("個股新聞看漲比率 ", total_k)
    global s
    s = "個股新聞看漲比率" + str(int(total_k))
    # print("個股新聞看跌比率", 100-total_k)
    global b
    b = "個股新聞看跌比率" + str(int(100-total_k))
    
    total_list_k= [total_k]
    total_list_n_k= [100-total_k]
    data_total_k = list(zip(total_list_k,total_list_n_k))
    df_total_k = pd.DataFrame(data_total_k, columns=['看漲','看跌'])


    names = ['看漲','看跌']
    # 轉為 dictionary
    data_total_k=dict(df_total_k[['看漲','看跌']].transpose())

    labels_total_k = list(data_total_k.keys())

    data_total_k = np.array(list(data_total_k.values()))   
    # 各筆資料總合，因為點計算百分比
    data_total_cum_k = data_total_k.cumsum(axis=1)
    category_total_colors_k = plt.get_cmap('RdYlGn')(np.linspace(0.15, 0.9, data_total_k.shape[1]))

    fig, ax = plt.subplots(figsize=(20, 1)) # 圖區大小(寬，高)
    ax.set_yticks(range(len(labels_total_k)))
    ax.set_yticklabels(range(len(labels_total_k)))# y 刻度標籤
    # y 由小到大，方向調整
    ax.invert_yaxis()
    # x 不出現刻度_k
    ax.xaxis.set_visible(False)
    # 設定 x 範圍，讓b1,b2,b3 加總後，都一樣大
    ax.set_xlim(0, np.sum(data_total_k, axis=1).max())

    for i, (name, color) in enumerate(zip(names, category_total_colors_k)):
        widths_total_k = data_total_k[:, i]
        starts_total_k = data_total_cum_k[:, i] - widths_total_k
        ax.barh(labels_total_k,  widths_total_k, left=starts_total_k, height=1,label=name, color=color)
        xcenters = starts_total_k + widths_total_k / 2
        # 讀取顏色資訊
        r, g, b, _ = color
        # 決定文字顏色，有兩種，白色與灰色
        text_color = 'white' if r * g * b < 0.5 else 'black'
        # 加入文字
        for y, (x, c) in enumerate(zip(xcenters, widths_total_k)):
            ax.text(x, y, str(int(c))+"%", ha='center', va='center',color=text_color, fontsize=18)#圖形內文字設定
    plt.title('個股新聞漲跌比率',loc='center',fontsize=20)
    ax.legend(ncol=len(names), bbox_to_anchor=(0, 1), loc='lower left', fontsize=10) #bbox_to_anchor圖例高度
    plt.tight_layout()
    plt.show

In [9]:
def customized2(): 
    TWA00_comment_list = [total_comments1]
    TWA00_comment_list_n = [100-total_comments1]
    data_TWA00_comment = list(zip(TWA00_comment_list,TWA00_comment_list_n))
    df_TWA00_comment = pd.DataFrame(data_TWA00_comment, columns=['看漲','看跌']) 
    names = ['看漲','看跌']
    # 轉為 dictionary
    data_TWA00_comment=dict(df_TWA00_comment[['看漲','看跌']].transpose())
    # dictionary 取 key 為 labels，即 0 到 2
    labels_TWA00_comment = list(data_TWA00_comment.keys())
    data_TWA00_comment = np.array(list(data_TWA00_comment.values()))
    # 各筆資料總合，因為點計算百分比
    data_TWA00_comment_cum = data_TWA00_comment.cumsum(axis=1)
    category_TWA00_comment_colors = plt.get_cmap('RdYlGn')(np.linspace(0.15, 0.9, data_TWA00_comment.shape[1]))

    fig, ax = plt.subplots(figsize=(20, 1)) # 圖區大小(寬，高)

    ax.set_yticks(range(len(labels_TWA00_comment)))

    ax.set_yticklabels(range(len(labels_TWA00_comment)))
    # y 由小到大，方向調整
    ax.invert_yaxis()
    # x 不出現刻度
    ax.xaxis.set_visible(False)
    # 設定 x 範圍，讓b1,b2,b3 加總後，都一樣大
    ax.set_xlim(0, np.sum(data_TWA00_comment, axis=1).max())
    for i, (name, color) in enumerate(zip(names,category_TWA00_comment_colors)):
        # 寬度，即變數值
        widths_TWA00_comment = data_TWA00_comment[:, i]
        starts_TWA00_comment = data_TWA00_comment_cum[:, i] - widths_TWA00_comment
        ax.barh(labels_TWA00_comment,  widths_TWA00_comment, left=starts_TWA00_comment, height=1,label=name, color=color)
        xcenters = starts_TWA00_comment + widths_TWA00_comment /2
        # 讀取顏色資訊
        r, g, b, _ = color
        # 決定文字顏色，有兩種，白色與灰色
        text_color = 'white' if r * g * b < 0.5 else 'black'
        # 加入文字
        for y, (x, c) in enumerate(zip(xcenters, widths_TWA00_comment)):
            ax.text(x, y, str(int(c))+"%", ha='center', va='center',color=text_color, fontsize=18)#圖形內文字設定
        # 增加圖例     
    plt.title('大盤言論風向漲跌比率',loc='center',fontsize=20,)
    ax.legend(ncol=len(names), bbox_to_anchor=(0, 1), loc='lower left', fontsize=10) #bbox_to_anchor圖例高度
    plt.tight_layout()
    plt.show

In [10]:
stock_num = ['1101', '1102', '1103', '1104', '1108', '1109', '1110', '1201', '1203', '1210', '1213', '1215', '1216', '1217', '1218', '1219', '1220', '1225', '1227', '1229', '1231', '1232', '1233', '1234', '1235', '1236', '1256', '1301', 
'1303', '1304', '1305', '1307', '1308', '1309', '1310', '1312', '1313', '1314', '1315', '1316', '1319', '1321', '1323', '1324', '1325', '1326', '1337', '1338', '1339', '1340', '1341', '1342', '1402', '1409', '1410', '1413', '1414', '1416', '1417', '1418', '1419', '1423', '1432', '1434', '1435', '1436', '1437', '1438', '1439', '1440', '1441', '1442', '1443', '1444', '1445', '1446', '1447', '1449', '1451', '1452', '1453', '1454', '1455', '1456', '1457', '1459', '1460', '1463', '1464', '1465', '1466', '1467', '1468', '1470', '1471', '1472', '1473', '1474', '1475', '1476', '1477', '1503', '1504', 
'1506', '1507', '1512', '1513', '1514', '1515', '1516', '1517', '1519', '1521', '1522', '1524', '1525', '1526', '1527', '1528', '1529', '1530', '1531', '1532', '1533', '1535', '1536', '1537', '1538', '1539', '1540', '1541', '1558', '1560', '1568', '1582', '1583', '1587', '1589', '1590', '1592', '1598', '1603', '1604', '1605', '1608', '1609', '1611', '1612', '1614', '1615', '1616', '1617', '1618', '1626', '1701', '1702', '1707', '1708', '1709', '1710', '1711', '1712', '1713', '1714', '1717', '1718', '1720', '1721', '1722', '1723', '1724', '1725', '1726', '1727', '1730', '1731', '1732', '1733', 
'1734', '1735', '1736', '1737', '1760', '1762', '1773', '1776', '1783', '1786', '1789', '1795', '1802', '1805', '1806', '1808', '1809', '1810', '1817', '1903', '1904', '1905', '1906', '1907', '1909', '2002', '2006', '2007', '2008', '2009', '2010', '2012', '2013', '2014', '2015', '2017', '2020', '2022', '2023', '2024', '2025', '2027', '2028', '2029', '2030', '2031', '2032', '2033', '2034', '2038', '2049', '2059', '2062', '2069', '2101', '2102', '2103', '2104', '2105', '2106', '2107', '2108', '2109', '2114', '2115', '2201', '2204', '2206', '2207', '2208', '2227', '2228', '2231', '2233', '2236', 
'2239', '2241', '2243', '2247', '2301', '2302', '2303', '2305', '2308', '2312', '2313', '2314', '2316', '2317', '2321', '2323', '2324', '2327', '2328', '2329', '2330', '2331', '2332', '2337', '2338', '2340', '2342', '2344', '2345', '2347', '2348', '2349', '2351', '2352', '2353', '2354', '2355', '2356', '2357', '2358', '2359', '2360', '2362', '2363', '2364', '2365', '2367', '2368', '2369', '2371', '2373', '2374', '2375', '2376', '2377', '2379', '2380', '2382', '2383', '2385', '2387', '2388', '2390', '2392', '2393', '2395', '2397', '2399', '2401', '2402', '2404', '2405', '2406', '2408', '2409', 
'2412', '2413', '2414', '2415', '2417', '2419', '2420', '2421', '2423', '2424', '2425', '2426', '2427', '2428', '2429', '2430', '2431', '2433', '2434', '2436', '2438', '2439', '2440', '2441', '2442', '2443', '2444', '2448', '2449', '2450', '2451', '2453', '2454', '2455', '2456', '2457', '2458', '2459', '2460', '2461', '2462', '2464', '2465', '2466', '2467', '2468', '2471', '2472', '2474', '2476', '2477', '2478', '2480', '2481', '2482', '2483', '2484', '2485', '2486', '2488', '2489', '2491', '2492', '2493', '2495', '2496', '2497', '2498', '2501', '2504', '2505', '2506', '2509', '2511', '2514', 
'2515', '2516', '2520', '2524', '2527', '2528', '2530', '2534', '2535', '2536', '2537', '2538', '2539', '2540', '2542', '2543', '2545', '2546', '2547', '2548', '2597', '2601', '2603', '2605', '2606', '2607', '2608', '2609', '2610', '2611', '2612', '2613', '2614', '2615', '2616', '2617', '2618', '2630', '2633', '2634', '2636', '2637', '2642', '2701', '2702', '2704', '2705', '2706', '2707', '2712', '2722', '2723', '2727', '2731', '2739', '2748', '2801', '2809', '2812', '2816', '2820', '2823', '2832', '2834', '2836', '2838', '2841', '2845', '2849', '2850', '2851', '2852', '2855', '2867', '2880', 
'2881', '2882', '2883', '2884', '2885', '2886', '2887', '2888', '2889', '2890', '2891', '2892', '2897', '2901', '2903', '2904', '2905', '2906', '2908', '2910', '2911', '2912', '2913', '2915', '2923', '2929', '2936', '2939', '3002', '3003', '3004', '3005', '3006', '3008', '3010', '3011', '3013', '3014', '3015', '3016', '3017', '3018', '3019', '3021', '3022', '3023', '3024', '3025', '3026', '3027', '3028', '3029', '3030', '3031', '3032', '3033', '3034', '3035', '3036', '3037', '3038', '3040', '3041', '3042', '3043', '3044', '3045', '3046', '3047', '3048', '3049', '3050', '3051', '3052', '3054', 
'3055', '3056', '3057', '3058', '3059', '3060', '3062', '3090', '3094', '3130', '3149', '3164', '3167', '3189', '3209', '3229', '3231', '3257', '3266', '3296', '3305', '3308', '3311', '3312', '3321', '3338', '3346', '3356', '3376', '3380', '3383', '3406', '3413', '3416', '3419', '3432', '3437', '3443', '3450', '3454', '3481', '3494', '3501', '3504', '3515', '3518', '3528', '3530', '3532', '3533', '3535', '3536', '3543', '3545', '3550', '3557', '3563', '3576', '3583', '3588', '3591', '3593', '3596', '3605', '3607', '3617', '3622', '3645', '3653', '3661', '3665', '3669', '3673', '3679', '3682', 
'3686', '3694', '3698', '3701', '3702', '3703', '3704', '3705', '3706', '3708', '3711', '3712', '4104', '4106', '4108', '4119', '4133', '4137', '4141', '4142', '4148', '4155', '4164', '4190', '4306', '4414', '4426', '4438', '4439', '4526', '4532', '4536', '4540', '4545', '4551', '4552', '4555', '4557', '4560', '4562', '4564', '4566', '4571', '4572', '4576', '4581', '4720', '4722', '4725', '4737', '4739', '4746', '4755', '4763', '4764', '4766', '4807', '4904', '4906', '4912', '4915', '4916', '4919', '4927', '4930', '4934', '4935', '4938', '4942', '4943', '4952', '4956', '4958', '4960', '4961', 
'4967', '4968', '4976', '4977', '4989', '4994', '4999', '5007', '5203', '5215', '5225', '5234', '5243', '5258', '5264', '5269', '5283', '5284', '5285', '5288', '5388', '5434', '5469', '5471', '5484', '5515', '5519', '5521', '5522', '5525', '5531', '5533', '5534', '5538', '5546', '5607', '5608', '5706', '5871', '5876', '5880', '5906', '5907', '6005', '6024', '6108', '6112', '6115', '6116', '6117', '6120', '6128', '6131', '6133', '6136', '6139', '6141', '6142', '6152', '6153', '6155', '6164', '6165', '6166', '6168', '6172', '6176', '6177', '6183', '6184', '6189', '6191', '6192', '6196', '6197', 
'6201', '6202', '6205', '6206', '6209', '6213', '6214', '6215', '6216', '6224', '6225', '6226', '6230', '6235', '6239', '6243', '6251', '6257', '6269', '6271', '6277', '6278', '6281', '6282', '6283', '6285', '6288', '6289', '6405', '6409', '6412', '6414', '6415', '6416', '6431', '6442', '6443', '6449', '6451', '6452', '6456', '6464', '6477', '6491', '6504', '6505', '6525', '6531', '6533', '6541', '6552', '6558', '6573', '6579', '6581', '6582', '6591', '6592', '6598', '6605', '6625', '6641', '6655', '6666', '6668', '6669', '6670', '6671', '6672', '6674', '6698', '6706', '6715', '6754', '8011', 
'8016', '8021', '8028', '8033', '8039', '8046', '8070', '8072', '8081', '8101', '8103', '8104', '8105', '8110', '8112', '8114', '8131', '8150', '8163', '8201', '8210', '8213', '8215', '8222', '8249', '8261', '8271', '8341', '8367', '8374', '8404', '8411', '8422', '8427', '8429', '8442', '8443', '8454', '8462', '8463', '8464', '8466', '8467', '8473', '8478', '8480', '8481', '8482', '8488', '8499', '8926', '8940', '8996', '9802', '9902', '9904', '9905', '9906', '9907', '9908', '9910', '9911', '9912', '9914', '9917', '9918', '9919', '9921', '9924', '9925', '9926', '9927', '9928', '9929', '9930', 
'9931', '9933', '9934', '9935', '9937', '9938', '9939', '9940', '9941', '9942', '9943', '9944', '9945', '9946', '9955', '9958']
def check_s_n(num):
    if num not in stock_num:
        return 0
    else:
        return 1

In [ ]:
from tkinter import ttk
window = tk.Tk()
# roots = tk.Tk()
# 設定視窗標題、大小和背景顏色
window.title('台股天氣預報機器人')
window.geometry('800x600')
window.configure(background='white') # 背景顏色
# 標題與介紹
header_label = tk.Label(window, text='今日a台股天氣~你想怎麼看??' , font=('Times', 24 , 'bold'), bg = "white" )
header_label.place(x = 160, y = 10 ,width=500, height=35) # y = 10
introduction = "歡迎來到台股天氣預報機器人\n請選擇下列的功能!"
introduction_label = tk.Label(window ,text = introduction,font = 15, bg = 'white')
introduction_label.place(x = 150, y = 45 ,width = 500, height = 65)

# 第一頁功能區 預設高度 = 150 
# 大盤預測  間隔 = 75
### 大盤 資料輸出視窗
def x_checker (): # 檢察權重指數
    x = float(taiex_entry.get())
    if  x > 1 or x <= 0:
        global words
        words = "權數不可小於0或大於1"
        root = tk.Tk()
        root.geometry("400x300")
        root.title("今日大盤預測結果為") 
        r_label = tk.Label(root, text = words,font = 12)
        r_label.pack()
        root.mainloop()
    else:
        taiex_data_getter(x)   
def taiex_data_getter(x): # 取得大盤預測的所有資料 + 分析    
    plot_TAIEX(x)   
    plot_TAIEX_1(x)
high1 = 110
taiex_label = tk.Label(window, text='台股大盤趨勢分析' , font=('微軟黑正體', 15) )
taiex_label.place(x = 200, y = high1 , width=400, height=35) 
taiex_intro = tk.Label(window, text='請輸入留言預測權數\n(必須介於0與1之間) : ' , font=('微軟黑正體', 8),bg = "white")
taiex_intro.place(x = 150 ,y = high1 + 50, width= 250, height=25 )
taiex_entry = tk.Entry(window)
taiex_entry.place(x = 350 ,y = high1 + 50, width= 100, height=15)
taiex_btn = tk.Button(window , text = "馬上分析" , font = ('微軟黑正體', 8), bg = "silver",state = tk.NORMAL, command = x_checker)
taiex_btn.place(x = 360, y = high1 + 75  , width=80, height=15)

# 個股預測 間隔100
high2 = high1 + 100
indstock_label = tk.Label(window, text='個股輿論趨勢分析' , font=('微軟黑正體', 15))
indstock_label.place(x = 200, y = high2 , width=400, height=35) 
s = ""
b = ""
## 個股查詢 資料輸出視窗
def ind_stock_printer(): # 此為測試爬蟲之用 尚未安裝分類器
#     window=tk.Tk()
#     window.title("Title")
#     window.geometry('800x600') #set a 800*600 window
#     window.resizable(0, 0) #fix the size of window
#     window.mainloop()
    stocknum = indstock_entry.get() # 要寫在裡面 才會每次都抓新的
    # root.title("個股預測結果")test
    check =  check_s_n(stocknum)# 檢視是否有輸入空的股票編號或是
    if check == 0: 
        ind_words = "編號輸入錯誤，無法進行分析"
        root = tk.Tk()
        r_label_1 = tk.Label(root , text = ind_words)
        r_label_1.pack()
        root.mainloop()
    else:
        # root.title("您的個股分析結果為")
         ind_stock_predictor(stocknum)
         openwin()

# 個股輸入按鈕
indstock_label = tk.Label(window, text = '股價編號' )
indstock_label.place(x = 340, y = high2 + 40  , width= 125, height=25)
indstock_entry = tk.Entry(window)
indstock_entry.place(x = 340, y = high2 + 58  , width= 125, height=25)
indstock_btn = tk.Button(window , text = "馬上分析" , font = ('微軟黑正體', 8), bg = "silver",state = tk.NORMAL,command = ind_stock_printer)
indstock_btn.place(x = 360, y = high2 + 100  , width=80, height=15)

#客製化大盤趨勢

# # 進階功能標題
pro_label = tk.Label(window, text='客製化您的台股天氣預報' , font=('Times', 15 , 'bold')  )
pro_label.place(x=200, y=360, width=400, height=35) 
combox = ttk.Combobox(window, 
                            values=[
                                    "財經新聞", 
                                    "股民留言",
                                    ],
                            state="normal")
combox.place(x=200, y=400)

# ### 輸出結果
# TWA00_title = ""
# TWA00_title_minus = ""
# total_comments1 = ""
# total_comments1_minus = ""

def customized_printer():
    # 利用字典儲存要參考的客製化設定
    customized = combox.get()
    # print(customized)
    if customized == "財經新聞":
         customized1()
    elif customized == "股民留言":
         customized2()
#     if customized == "財經新聞 + 股民留言":
#          customized.run_customized()
# 客製化按鈕
cm_btn = tk.Button(window , text = "馬上分析" , font = ('Times', 8), bg = "silver",command = customized_printer)
cm_btn.place(x = 400, y = 405  , width=80, height=15)

window.mainloop()

Building prefix dict from C:\My python\dict_all.txt ...
Loading model from cache C:\Users\apz33\AppData\Local\Temp\jieba.u00c367b986563c0985937721e33bcd72.cache
Loading model cost 0.05214262008666992 seconds.
Prefix dict has been built succesfully.
Building prefix dict from C:\My python\dict_all.txt ...
Loading model from cache C:\Users\apz33\AppData\Local\Temp\jieba.u00c367b986563c0985937721e33bcd72.cache
Loading model cost 0.03143715858459473 seconds.
Prefix dict has been built succesfully.
Building prefix dict from C:\My python\dict_all.txt ...
Loading model from cache C:\Users\apz33\AppData\Local\Temp\jieba.u00c367b986563c0985937721e33bcd72.cache
Loading model cost 0.03193306922912598 seconds.
Prefix dict has been built succesfully.
